In [ ]:
import pandas as pd
from IPython.display import display

In [25]:
path = "../data/cleaned_sample_superstore.xlsx"
df = pd.read_excel(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5009 entries, 0 to 5008
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      5009 non-null   object        
 1   order_date    5009 non-null   datetime64[ns]
 2   customer_id   5009 non-null   object        
 3   sales_amount  5009 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 156.7+ KB


In [9]:
# تبدیل ستون به datetime و ایجاد ستون دوره‌ای روزانه
df['order_date'] = pd.to_datetime(df['order_date'])
# df['order_period'] = df['order_date'].dt.to_period('Q')

# جایگزین: قرار دادن order_date به عنوان ایندکس و تبدیل ایندکس به PeriodIndex
# df['order_date'] = pd.to_datetime(df['order_date'])
# df = df.set_index('order_date')
# df.index = df.index.to_period('D')

df.dtypes


order_id                object
order_date      datetime64[ns]
customer_id             object
sales_amount           float64
dtype: object

In [23]:
# aggregate to customer level
rfm = df.groupby("customer_id").agg(
    recency   = ("order_date", lambda x: (df["order_date"].max() - x.max()).days),
    frequency = ("order_id",   "nunique"),
    monetary  = ("sales_amount","sum"),
).reset_index()

# scoring
rfm['recency_score']   = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
rfm['frequency_score'] = pd.qcut(rfm['frequency'], 5, labels=[1,2,3,4,5])
rfm['monetary_score']  = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])

# RFM Score
rfm['RFM_Score'] = (
    rfm['recency_score'].astype(str) +
    rfm['frequency_score'].astype(str) +
    rfm['monetary_score'].astype(str)
)

rfm.drop(columns=[
        'recency', 'frequency', 'monetary',
        'recency_score', 'frequency_score', 'monetary_score'
        ], inplace=True)

rfm

,customer_id,RFM_Score
0,AA-10315,225
1,AA-10375,552
2,AA-10480,113
3,AA-10645,335
4,AB-10015,111
...,...,...
788,XP-21865,453
789,YC-21895,525
790,YS-21880,545
791,ZC-21910,355


In [ ]:
# freq = "Y"  # 'all', 'M', 'Q', 'Y'

df = df.copy()

# ---- Calendar generator ----

# def compute_rfm(df, freq="all"):
#     """
#     freq: 'all', 'M', 'Q', 'Y'
#     """



# # ---- Aggregate: customer x period ----
# rfm = df.groupby(["customer_id", "period"]).agg(
#     last_order=("order_date", "max"),
#     frequency=("order_id", "nunique"),
#     monetary=("sales_amount", "sum")
# ).reset_index()

# # ---- Attach correct period_end ----
# rfm = rfm.merge(period_end, on="period", how="left")

# # ---- Recency calculation (always correct) ----
# rfm["recency"] = (rfm["period_end"] - rfm["last_order"]).dt.days





    # return rfm


rfm

,customer_id,period,frequency,monetary,recency,RFM_Score
0,AA-10315,2014,2,756.048,107,33334
1,AA-10315,2015,1,26.960,88,311
2,AA-10315,2016,1,4406.072,303,115
3,AA-10315,2017,1,374.480,184,112
4,AA-10375,2014,2,50.792,68,33341
...,...,...,...,...,...,...
2494,ZC-21910,2016,3,3215.733,267,15685
2495,ZC-21910,2017,4,227.066,54,36412
2496,ZD-21925,2014,1,244.760,126,23323
2497,ZD-21925,2016,3,1187.744,176,15694


In [90]:
# Customer Query! 
df['order_period'] = df['order_date'].dt.to_period('Y')

df[
    (df['customer_id'] == 'EP-13915') &
    (df['order_period'] == '2016')
]

df

,order_id,order_date,customer_id,sales_amount,order_period,period
1907,CA-2016-113390,2016-10-11,EP-13915,33.0640,2016,2016
2332,CA-2016-140641,2016-10-28,EP-13915,38.1900,2016,2016
4566,US-2016-119298,2016-11-25,EP-13915,59.9800,2016,2016
4594,US-2016-131611,2016-11-05,EP-13915,2007.9048,2016,2016
4688,US-2016-159856,2016-10-18,EP-13915,307.9200,2016,2016
4990,US-2017-165953,2016-12-31,EP-13915,71.0960,2016,2016


باشه، پس می‌خوای **بدون ساخت کلاس‌های جداگانه برای استراتژی‌ها**،
یک **کلاس RFMAnalyzer ساده و تمیز** داشته باشی
و فقط داخلش چند حالت (`classic`, `weighted`, `kmeans`) رو بیلت-این هندل کنی.

و همچنین بفهمی کدوم ساختار بهتره:

* یک کلاس اصلی
* یک فانکشن wrapper
* جای پلات‌ها کجاست؟

کاملاً شفاف برات می‌نویسم.

---

# ✅ جواب کوتاه و دقیق:

### **بله — باید یک کلاس RFMAnalyzer داشته باشی، و یک فانکشن wrapper که از این کلاس استفاده کنه.**

و **پلات‌ها باید خارج از کلاس** باشن (یا نهایتاً در یک ماژول جدا مثل `plots.py`).

این بهترین معماریه.

---

# 🧠 چرا این ساختار بهترینه؟

## 1) کلاس RFMAnalyzer فقط کارهای “دیتا” رو انجام می‌ده:

* آماده‌سازی period
* محاسبه RFM
* اسکوردهی
* سگمنت کردن
* ساخت transition matrix
* خروجی نهایی (dataframes)

### ❌ اما هیچ پلاتی تولید نمی‌کنه!

پلات‌ها باید در جای دیگری باشند.

---

## 2) پلات‌ها معمولاً نیاز به Streamlit, Matplotlib, Plotly دارند

و این‌ها نباید داخل `RFMAnalyzer` باشن چون:

* کلاس دیتا سنگین می‌شود
* dependencyها قاطی می‌شوند
* قابل تست نیست
* بعداً سخت توسعه پیدا می‌کند

بنابراین پلات‌ها = **ماژول جدا**.

---

## 3) یک فانکشن wrapper برای انتخاب mode (`classic`, `weighted`, `kmeans`)

به‌جای Strategy Pattern، همین‌جا یک سوئیچ می‌گذاری.

کاملاً OK.

---

# ⚙️ معماری پیشنهادی (ساده، تمیز، قابل توسعه)

```
/rfm/
    rfm_analyzer.py      → کلاس اصلی
    rfm_modes.py         → فانکشن انتخاب حالت
    rfm_plots.py         → پلات‌ها
    rfm_utils.py         → توابع کمکی
```

---

# 🟦 1) ساخت کلاس RFMAnalyzer

این کلاس فقط “منطق مشترک” دارد.

مثلاً:

```
class RFMAnalyzer:
    def __init__(self, df, date_col, customer_col, amount_col):
        # ست کردن پایه‌ها

    def make_period(self, freq):
        # ساخت period

    def calculate_rfm(self):
        # محاسبه recency + freq + monetary

    def score_classic(self):
        # quantile scoring

    def score_weighted(self):
        # weighted scoring

    def score_kmeans(self):
        # k-means

    def segment_classic(self):
        # marketing labels

    def segment_weighted(self):
        # numeric 111-555

    def segment_kmeans(self):
        # cluster labels

    def transition_matrix(self):
        # ساخت matrix

    def run(self, mode):
        # اجرای کامل pipeline
```

در متد `run` می‌زنی:

```
if mode == "classic":
    self.score_classic()
    self.segment_classic()

elif mode == "weighted":
    self.score_weighted()
    self.segment_weighted()

elif mode == "kmeans":
    self.score_kmeans()
    self.segment_kmeans()
```

همه چیز داخلی و ساده.

---

# 🟩 2) فانکشن wrapper

این فانکشن کاربر استفاده می‌کند:

```
def analyze_rfm(df, mode="classic"):
    analyzer = RFMAnalyzer(df, "order_date", "customer_id", "sales_amount")
    result = analyzer.run(mode)
    return result
```

این باعث می‌شود رابط کاربری (API Usage) خیلی تمیز باشد:

```
history, segments, transition = analyze_rfm(df, mode="weighted")
```

---

# 🟧 3) پلات‌ها → فایل جدا (`rfm_plots.py`)

این مهم است:

❌ پلات‌ها را داخل analyzer ننویس
✔ بیرون بنویس و تنها خروجی آن‌ها dataframe باشد.

مثال:

```
def plot_segment_distribution(segments_df):
    # یک پلات بساز و ریترن کن

def plot_transition_heatmap(transition_matrix):
    # heatmap
```

و در پروژه Streamlit:

```
history, segments, transition = analyze_rfm(df, mode="classic")

plot_segment_distribution(segments)
plot_transition_heatmap(transition)
```

---

# 🎯 نتیجه نهایی معماری

خیلی ساده، تمیز، قابل‌فهم:

```
RFMAnalyzer  → فقط منطق دیتا
rfm_modes    → مدها و سوییچ بینشون
rfm_plots    → تمام پلات‌ها جدا
wrapper      → analyze_rfm()
```


# RFM

In [1]:
import sys, os, importlib

# Go one folder up from temp_notebooks to project root
sys.path.append(os.path.abspath(".."))

from core.rfm import rfm_analysis
from utils.data_loader import DataLoader

In [23]:
# ========== RUN FUNCTION ==========
data = DataLoader().sample_data()

rfm_result = rfm_analysis(data=data, type='classic', freq='Q')
print(rfm_result.head())


  customer_id  period rfm_score        segment
0    AA-10315  2014Q1       515           Lost
1    AA-10315  2014Q3       411  New Customers
2    AA-10315  2015Q4       111  New Customers
3    AA-10315  2016Q1       315  New Customers
4    AA-10315  2017Q2       514           Lost


In [3]:
data

,order_id,order_date,customer_id,sales_amount
0,CA-2014-100006,2014-09-07,DK-13375,377.970
1,CA-2014-100090,2014-07-08,EB-13705,699.192
2,CA-2014-100293,2014-03-14,NF-18475,91.056
3,CA-2014-100328,2014-01-28,JC-15340,3.928
4,CA-2014-100363,2014-04-08,JM-15655,21.376
...,...,...,...,...
5004,US-2017-168802,2017-11-03,JO-15145,18.368
5005,US-2017-169320,2017-07-23,LH-16900,171.430
5006,US-2017-169488,2017-09-07,AA-10375,56.860
5007,US-2017-169502,2017-08-28,MG-17650,113.410


# analysis of rfm

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [26]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

periods = sorted(rfm_result['period'].unique(), key=lambda p: str(p))
n = len(periods)
cols = min(n, n)
rows = (n + cols - 1) // cols

fig = make_subplots(rows=rows, cols=cols,
                    specs=[[{'type': 'domain'}]*cols for _ in range(rows)],
                    subplot_titles=[str(p) for p in periods])

for i, period in enumerate(periods):
    dfp = rfm_result[rfm_result['period'] == period]
    counts = dfp['segment'].value_counts()
    row = i // cols + 1
    col = i % cols + 1
    fig.add_trace(
        go.Pie(labels=counts.index, values=counts.values, name=str(period), sort=False),
        row=row, col=col
    )

fig.update_layout(height=350*rows, width=420*cols, title_text="RFM Segment Distribution — All Periods")
fig.add_layout_image(
    dict(
        source="https://vizos.ai/wp-content/uploads/2024/05/cropped-VizOs-Logo-1.png",
        xref="paper", yref="paper",
        x=1, y=0,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

In [33]:
chart = px.treemap(rfm_result, path=['period', 'segment'],
           title='RFM Segment Treemap by Period')
chart.show()

In [38]:
data

,order_id,order_date,customer_id,sales_amount
0,CA-2014-100006,2014-09-07,DK-13375,377.970
1,CA-2014-100090,2014-07-08,EB-13705,699.192
2,CA-2014-100293,2014-03-14,NF-18475,91.056
3,CA-2014-100328,2014-01-28,JC-15340,3.928
4,CA-2014-100363,2014-04-08,JM-15655,21.376
...,...,...,...,...
5004,US-2017-168802,2017-11-03,JO-15145,18.368
5005,US-2017-169320,2017-07-23,LH-16900,171.430
5006,US-2017-169488,2017-09-07,AA-10375,56.860
5007,US-2017-169502,2017-08-28,MG-17650,113.410


In [45]:
data.groupby('order_date').sum().reset_index()

,order_date,order_id,customer_id,sales_amount
0,2014-01-03,CA-2014-103800,DP-13000,16.4480
1,2014-01-04,CA-2014-112326,PO-19195,288.0600
2,2014-01-05,CA-2014-141817,MB-18085,19.5360
3,2014-01-06,CA-2014-106054CA-2014-130813CA-2014-167199,JO-15145LS-17230ME-17320,4407.1000
4,2014-01-07,CA-2014-105417,VS-21820,87.1580
...,...,...,...,...
1232,2017-12-26,CA-2017-112487CA-2017-121741CA-2017-136238US-2...,TC-21535YC-21895KB-16240PO-18850,814.5940
1233,2017-12-27,CA-2017-134404,DR-12880,177.6360
1234,2017-12-28,CA-2017-101322CA-2017-122798CA-2017-127516CA-2...,JG-15310SV-20935EH-14005HM-14860CS-12400GH-146...,1657.3508
1235,2017-12-29,CA-2017-118885CA-2017-130631CA-2017-146626CA-2...,JG-15160BS-11755BP-11185KB-16600MC-17845KH-16360,2915.5340


In [ ]:
chart =px.line(data.groupby('order_date').sum().reset_index(), x='order_date', y='sales_amount').show()